# [Finetuning falcon-40b with Axolotl+QLoRA](https://wandb.ai/utensil/falcon-qlora/reports/Finetuning-falcon-7b-with-Axolotl-QLoRA--Vmlldzo0NTIzNTMw)

<!-- https://jupyterlab.readthedocs.io/en/stable/user/commands.html#commands-list -->
<button data-commandLinker-command="apputils:change-theme" data-commandlinker-args='{"theme": "JupyterLab Dark"}' href="#">Dark theme</button>
<button data-commandLinker-command="filebrowser:go-to-path" data-commandlinker-args='{"path": "/workspace/llm-playground/"}' href="#">llm-playground</button>
<button data-commandLinker-command="filebrowser:go-to-path" data-commandlinker-args='{"path": "/workspace/axolotl/"}' href="#">axolotl</button>
<button data-commandLinker-command="filebrowser:go-to-path" data-commandlinker-args='{"path": "/workspace/llm-playground/notebooks/axolotl/runpod"}' href="#">Runpod notebooks</button>
<button data-commandLinker-command="filebrowser:go-to-path" data-commandlinker-args='{"path": "/workspace/axolotl/examples"}' href="#">axolotl configs</button>
<button data-commandLinker-command="filebrowser:go-to-path" data-commandlinker-args='{"path": "/workspace/llm-playground/storage"}' href="#">Storage</button>
<button data-commandLinker-command="docmanager:open" data-commandlinker-args='{"path": "/workspace/axolotl/examples/falcon/qlora.yml"}' href="#">Edit config</button>

In [1]:
%cd /workspace/axolotl

/workspace/axolotl


In [2]:
!accelerate config --config_file configs/accelerate/default_config.yaml default

Setting ds_accelerator to cuda (auto detect)
accelerate configuration saved at /root/.cache/huggingface/accelerate/default_config.yaml


## #1  40b 4*2

In [3]:
!cat examples/falcon/qlora.yml

base_model: tiiuae/falcon-40b
base_model_config: tiiuae/falcon-40b
trust_remote_code: true
model_type: AutoModelForCausalLM
tokenizer_type: AutoTokenizer
load_in_8bit: false
load_in_4bit: true
gptq: false
strict: false
push_dataset_to_hub:
datasets:
  - path: QingyiSi/Alpaca-CoT
    data_files:
      - Chain-of-Thought/formatted_cot_data/gsm8k_train.json
    type: "alpaca:chat"
dataset_prepared_path: last_run_prepared
val_set_size: 0.01
adapter: qlora
lora_model_dir:
sequence_len: 2048
max_packed_sequence_len:
lora_r: 64
lora_alpha: 16
lora_dropout: 0.05
lora_target_modules:
lora_target_linear: true
lora_fan_in_fan_out:
wandb_project: falcon-qlora
wandb_watch:
wandb_run_id:
wandb_log_model:
output_dir: ./qlora-out
micro_batch_size: 4
gradient_accumulation_steps: 2
num_epochs: 3
optimizer: paged_adamw_32bit
torchdistx_path:
lr_scheduler: cosine
learning_rate: 0.0002
train_on_inputs: false
group_by_length: false
bf16: true
fp16: false
tf32: true
gradient_checkpointing: true
# stop traini

In [12]:
!accelerate launch scripts/finetune.py examples/falcon/qlora.yml

Setting ds_accelerator to cuda (auto detect)

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /root/miniconda3/envs/py3.9/lib/python3.9/site-packages/bitsandbytes-0.39.0-py3.9.egg/bitsandbytes/libbitsandbytes_cuda118.so
/root/miniconda3/envs/py3.9/lib/python3.9/site-packages/bitsandbytes-0.39.0-py3.9.egg/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib64'), PosixPath('/usr/local/nvidia/lib')}
  warn(msg)
/root/miniconda3/envs/py3.9/lib/python3.9/site-packages/bitsandbytes-0.39.0-py3.9.egg/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libc

## #2 


In [ ]:
!cat examples/falcon/qlora.yml

In [ ]:
!accelerate launch scripts/finetune.py examples/falcon/qlora.yml

## #3

In [ ]:
!cat examples/falcon/qlora.yml

In [ ]:
!accelerate launch scripts/finetune.py examples/falcon/qlora.yml

## #4

In [ ]:
!cat examples/falcon/qlora.yml

In [ ]:
!accelerate launch scripts/finetune.py examples/falcon/qlora.yml

## Below are ad hoc cells handling issues during training

### Force release VRAM

In [24]:
# First interupt the kernel, wait a few seconds then run this to kill finetune to release VRAM
!ps aux|grep python|grep finetune|awk '{print $2}'|xargs kill

### Clean the finetuned model and all checkpoints

In [28]:
# Only run this to start over
!rm -rf ./qlora-out

### Zip the prepared dataset

In [ ]:
!apt install zip
!zip -r last_run_prepared.zip -xi last_run_prepared

### Monitoring GPU

In [ ]:
# Run this in a seperate terminal
!nvitop -m full

### Fix DISK FULL

In [14]:
%cd /

/


In [ ]:
!du -d 2 -h|grep G

In [ ]:
!du -d 2 -h /root/.local

In [13]:
!rm -rf /root/.local/share/Trash/

In [ ]:
!rm -rf /root/.local/share/wandb/

In [ ]:
!rm -rf /root/.cache/wandb/

### Check who is using GPU

In [ ]:
!apt install lsof

In [ ]:
!lsof /dev/nvidia*

### Upload checkpoints to HF

In [ ]:
!cp -r /workspace/axolotl/qlora-out/checkpoint-120 /workspace/llm-playground/storage/falcon-qlora/

In [ ]:
!python /workspace/llm-playground/helper/storage.py -u

### Update axolotl

In [6]:
%cd /workspace/

/workspace


In [7]:
!git clone https://github.com/OpenAccess-AI-Collective/axolotl axolotl-update

Cloning into 'axolotl-update'...
remote: Enumerating objects: 2484, done.
remote: Counting objects: 100% (1234/1234), done.
remote: Compressing objects: 100% (356/356), done.
remote: Total 2484 (delta 907), reused 1082 (delta 812), pack-reused 1250
Receiving objects: 100% (2484/2484), 1.30 MiB | 3.02 MiB/s, done.
Resolving deltas: 100% (1539/1539), done.


In [8]:
!cp -r axolotl-update/* axolotl

In [9]:
%cd /workspace/axolotl

/workspace/axolotl


In [10]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   README.md
	modified:   configs/cerebras_1_3B_alpaca.yml
	modified:   configs/galactica_1_3B.yml
	modified:   configs/gpt_neox_20b.yml
	modified:   configs/llama_13B_alpaca.yml
	modified:   configs/llama_65B_alpaca.yml
	modified:   configs/llama_7B_4bit.yml
	modified:   configs/llama_7B_alpaca.yml
	modified:   configs/llama_7B_jeopardy.yml
	modified:   configs/pythia_1_2B_alpaca.yml
	modified:   configs/quickstart.yml
	modified:   configs/sample.yml
	modified:   configs/stability_3b.yml
	modified:   configs/vicuna_13B_4bit_reflect.yml
	modified:   docker/Dockerfile-base
	modified:   examples/gptq-lora-7b/config.yml
	modified:   examples/mpt-7b/config.yml
	modified:   requirements.txt
	modified:   scripts/finetune.py
	modified:   scripts/runpod-entrypoi

In [11]:
!pip install -e .

Obtaining file:///workspace/axolotl
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-install-w8_gs79m/transformers_d7bb90e3e98d4c5c855b5488e6abe26d
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-install-w8_gs79m/transformers_d7bb90e3e98d4c5c855b5488e6abe26d
  Resolved https://github.com/huggingface/transformers.git to commit f49a3453caa6fe606bb31c571423f72264152fce
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Attempting uninstall: axolotl
    Found existing installation: axolotl 0.1
    Uninstalling axolotl-0.1:
      Successfully uninstalled axolotl-0.1
  Running setup.py develop for axolotl


### A new bash without tmux etc.

In [ ]:
!bash --norc --noprofile

### Clean up all checkpoints but last one

In [ ]:
cd /workspace/axolotl/qlora-out/ && ls -lhta |grep checkpoint- | awk 'NR > 1 {print $9}' | xargs rm -rf